<h3 align="Center">
    <img alt="Logo" title="#logo" width="250px" src="https://www.insart.com/wp-content/uploads/Enova.png">
    <br>
</h3>

# <center> **Diamond prices model**
## <center> **Methodologic report**
### <center> **Analytics team**
### <center> Analysts:
### <center> *Kevin M. Figueroa*
### <center> *Dec 6th 2022*